# Задание 2.3
Напишите запрос для расчета общей суммы продаж за каждый месяц

(Представлены два способа решения)

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").getOrCreate()


API

In [ ]:
from pyspark.sql import functions as F

df_orders = spark.read.csv("CVS\\orders.csv", inferSchema=True, header=True, sep=",")
df_order_items = spark.read.csv("CVS\\order_items.csv", inferSchema=True, header=True, sep=",")

df_res = df_orders.join(df_order_items, 
                           df_order_items["order_id"] == df_orders["order_id"], "inner")


df_res = df_res.groupBy(F.month("order_date")
                        ).agg(
                        F.count("product_id").alias("Total Number of Orders"),
                        F.sum(df_order_items["quantity"] * 
                        (df_order_items["list_price"] * 
                         (1-df_order_items["discount"]))
                         ).alias("Total sales")).withColumnRenamed("month(order_date)", "Month")

df_res.orderBy("Month").show()

+-----+----------------------+------------------+
|Month|Total Number of Orders|       Total sales|
+-----+----------------------+------------------+
|    1|                   466| 882193.0074000009|
|    2|                   429| 669694.1313000012|
|    3|                   535| 853503.2238000015|
|    4|                   669| 1212356.824700001|
|    5|                   309| 473503.2470000006|
|    6|                   324|  589616.769000001|
|    7|                   315|440890.10700000083|
|    8|                   364| 524588.6468000006|
|    9|                   348| 575460.8305000005|
|   10|                   370| 526187.5179000009|
|   11|                   294|475269.32140000083|
|   12|                   299|465852.93080000085|
+-----+----------------------+------------------+



spark.sql

In [18]:
df_orders.createOrReplaceTempView("orders")
df_order_items.createOrReplaceTempView("order_items")

spark.sql("""
            SELECT month(orders.order_date) AS Month,
                    count(orders.order_id) AS Total_Number_of_Orders,
                    sum(order_items.quantity * (order_items.list_price * (1 - order_items.discount))) AS Total_Sales
            FROM orders
            inner join order_items on order_items.order_id = orders.order_id
            GROUP BY Month
            ORDER BY Month;
          """).show()

+-----+----------------------+------------------+
|Month|Total_Number_of_Orders|       Total_Sales|
+-----+----------------------+------------------+
|    1|                   466| 882193.0074000009|
|    2|                   429| 669694.1313000012|
|    3|                   535| 853503.2238000015|
|    4|                   669| 1212356.824700001|
|    5|                   309| 473503.2470000006|
|    6|                   324|  589616.769000001|
|    7|                   315|440890.10700000083|
|    8|                   364| 524588.6468000006|
|    9|                   348| 575460.8305000005|
|   10|                   370| 526187.5179000009|
|   11|                   294|475269.32140000083|
|   12|                   299|465852.93080000085|
+-----+----------------------+------------------+

